In [1]:
from langchain_chroma import Chroma
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_huggingface import HuggingFaceEmbeddings

In [2]:
from langchain_ollama import OllamaLLM
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [35]:
gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0
)


In [4]:
MODEL = "gemma:2b-instruct"
DB_NAME = "vector_db"


In [5]:
# Import libraries for RAG pipeline

import glob

from langchain_text_splitters import RecursiveCharacterTextSplitter


In [6]:
# Load all markdown files from data folder

markdown_files = glob.glob("data/*.md")

documents = []

for file_path in markdown_files:
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        documents.append({"content": content, "source": file_path})

print(f"Loaded {len(documents)} markdown files from data folder")

Loaded 16 markdown files from data folder


In [7]:
import shutil
import os

if os.path.exists(DB_NAME):
    shutil.rmtree(DB_NAME)
    print("Old DB deleted.")
else:
    print("No existing DB found.")


Old DB deleted.


In [8]:
# Split documents into chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

chunks = []
for doc in documents:
    doc_chunks = text_splitter.create_documents(
        texts=[doc["content"]],
        metadatas=[{"source": doc["source"]}]
    )
    chunks.extend(doc_chunks)

print(f"Created {len(chunks)} chunks from {len(documents)} documents")

Created 189 chunks from 16 documents


In [9]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=DB_NAME
    )

retriever = vectorstore.as_retriever()

print("New DB created successfully.")

New DB created successfully.


In [10]:

llm = OllamaLLM(temperature=0, model=MODEL)

In [11]:
SYSTEM_PROMPT_TEMPLATE = """
You are a strict intent classifier.

Classify the user query into EXACTLY one of these labels:

explanation
advisory
risk_assessment
simulation

Label meanings:

explanation = asking to define or explain something.
advisory = asking what action to take OR how to do something.
risk_assessment = asking if something is bad, harmful, or risky.
simulation = describing a specific hypothetical situation and asking what will happen.

Rules:
- Output ONLY one label.
- No extra words.
- No punctuation.
"""


## Local RAG

In [12]:
ALLOWED_LABELS = {
    "explanation",
    "advisory",
    "risk_assessment",
    "simulation"
}

def classify_intent(question: str):
    q = question.lower().strip()

    response = llm.invoke([
        SystemMessage(content=SYSTEM_PROMPT_TEMPLATE),
        HumanMessage(content=question)
    ])
    
    label = response.strip().lower()
    
    if label not in ALLOWED_LABELS:
        return "explanation"  
    return label


In [13]:
def route_question(question: str):

    intent = classify_intent(question)
    context = retrieve_context(question)

    if intent == "explanation":
        return handle_explanation(question, context)
        
    elif intent == "advisory":
        return handle_advisory(question, context)
        
    elif intent == "risk_assessment":
        return handle_risk_assessment(question, context)
        
    elif intent == "simulation":
        return handle_simulation(question, context)

    return intent


In [14]:
def retrieve_context(question: str, k: int = 4):
    docs = retriever.invoke(question)
    context = "\n\n".join([d.page_content for d in docs])
    return context


In [15]:
def handle_explanation(question: str, context: str):
    prompt = f"""
    You are a UK credit education assistant.

    Use the verified knowledge base context below as your primary source.

    Context:
    {context}

    Question:
    {question}

    Provide a structured explanation:

    - Clear Definition
    - Why It Matters
    - Practical Example (if relevant)

    Do NOT invent statistics or exact credit score numbers.
    If context does not contain the answer, say so clearly.
    """

    return llm.invoke(prompt)


In [16]:
def handle_risk_assessment(question: str, context: str):
    
    prompt = f"""
    You are a UK credit risk assessment engine.

    Use the following verified UK credit knowledge base context:

    {context}

    The user question is:
    {question}

    You will Estimate the following
    Risk Level:
    Baseline Reason: 

    Your task:
    - Validate or adjust the risk level if necessary.
    - Provide clear reasoning grounded in UK credit principles.
    - Do NOT invent numerical score changes.
    - Keep explanation concise and practical.

    Provide output in this format:

    - Risk Level (Low / Medium / High)
    - Why This Is Risky
    - Time Sensitivity (Immediate / Short-Term / Long-Term)
    - Recommended Precaution (1â€“2 steps)
    """

    return llm.invoke(prompt)

    



In [17]:
def handle_advisory(question: str, context: str):

    prompt = f"""
    You are a UK credit advisor.

    Use the following verified UK credit information:

    {context}

    Question:
    {question}

    Format:
    - Goal
    - 3 Action Steps
    - 1 Warning
    """

    return llm.invoke(prompt)


In [18]:
def handle_simulation(question: str, context: str):

    prompt = f"""
    You are a UK credit system simulation engine.

    Use the following verified UK credit knowledge base context:

    {context}

    The user is asking a what-if scenario.

    Question:
    {question}

    Provide structured output in this format:

    - Scenario Summary
    - Short-Term Impact (0â€“3 months)
    - Medium-Term Impact (3â€“12 months)
    - Estimated Risk Level (Low / Medium / High)
    - Recovery Strategy (3 steps)
    - Key Insight

    Use realistic UK principles.
    Do NOT hallucinate exact credit score numbers.
    """

    return llm.invoke(prompt)


## Fallback

In [19]:
ALLOWED_LABELS = {
    "explanation",
    "advisory",
    "risk_assessment",
    "simulation"
}

def classify_intent(question: str):
    q = question.lower().strip()

    response = llm.invoke([
        SystemMessage(content=SYSTEM_PROMPT_TEMPLATE),
        HumanMessage(content=question)
    ])
    
    label = response.strip().lower()
    
    if label not in ALLOWED_LABELS:
        return "explanation"  
    return label


In [20]:
def route_question(question: str):

    intent = classify_intent(question)

    context, confidence = retrieve_context(question, intent)

    decision = decide_execution(intent, confidence)

    print(f"[Intent: {intent}] [Confidence: {confidence:.3f}] [Decision: {decision}]")

    if decision == "local":

        if intent == "explanation":
            return handle_explanation(question, context)
        
        elif intent == "advisory":
            return handle_advisory(question, context)
        
        elif intent == "risk_assessment":
            return handle_risk_assessment(question, context)
        
        elif intent == "simulation":
            return handle_simulation(question, context)

    else:
        print(" Escalating to Gemini (Cloud) ")
        return handle_cloud_execution(question, intent)


In [21]:
def retrieve_context(question: str, intent: str):
    
    # Category-aware k
    if intent == "simulation":
        k = 8
    elif intent == "explanation":
        k = 6
    else:
        k = 4

    # Get docs with similarity scores
    docs_with_scores = vectorstore.similarity_search_with_score(question, k=k)

    docs = [doc for doc, score in docs_with_scores]
    scores = [score for doc, score in docs_with_scores]

    context = "\n\n".join([d.page_content for d in docs])

    # Confidence = inverse of distance (Chroma returns distance, lower is better)
    # Convert to similarity-style confidence
    if scores:
        confidence = max(0, 1 - scores[0])  # top match confidence
    else:
        confidence = 0

    return context, confidence


In [22]:
def decide_execution(intent: str, confidence: float):
    
    # Simple initial logic
    if confidence < 0.55:
        return "cloud"
    
    # Simulation queries require higher confidence
    if intent == "simulation" and confidence < 0.65:
        return "cloud"
    
    return "local"


In [23]:
def handle_cloud_execution(question: str, intent: str):

    prompt = f"""
    You are an expert UK credit advisor.

    The local system had low retrieval confidence.
    Provide a structured answer for this intent: {intent}

    Question:
    {question}

    Follow the same structured format used by the system.
    Do NOT fabricate exact score numbers.
    """

    response = gemini_llm.invoke(prompt)

    return response.content


In [ ]:
import sys
print(sys.executable)


c:\Users\sharv\OneDrive\Desktop\CreditSystem\.venv\Scripts\python.exe


In [ ]:
import sys
print(sys.executable)


c:\Users\sharv\OneDrive\Desktop\CreditSystem\.venv\Scripts\python.exe


In [36]:
print(route_question("How do AI-based risk models in UK banks work?"))


[Intent: simulation] [Confidence: -0.024] [Decision: cloud]
 Escalating to Gemini (Cloud) 
**Structured Answer: AI-Based Risk Models in UK Banks**

AI-based risk models are sophisticated tools employed by UK banks to enhance the accuracy, efficiency, and speed of risk assessment across various domains. They leverage advanced computational techniques to analyse vast datasets, providing deeper insights into potential risks than traditional methods.

**1. Core Purpose & Evolution:**
*   **Enhanced Risk Assessment:** Moving beyond traditional statistical models, AI provides more granular and dynamic risk profiling across credit, market, operational, and fraud risks.
*   **Proactive Management:** Enables banks to identify emerging risks and opportunities earlier, facilitating more timely interventions and strategic decision-making.
*   **Regulatory Compliance:** Supports adherence to stringent UK and international financial regulations (e.g., PRA, FCA, Basel III, IFRS 9).

**2. Key Data Sou